In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# Greedy Matrix Codenames

A first attempt at the greedy matrix approach to playing codenames as a Spymaster. Essentially, all ally word combinations of up to 5 are calculated and clues are generated for each by using Gensim's word similarity functions. The resulting clue words are compared with the rest of the board and, using weightings for card types, scores are generated.

For this first pass, we're using the conceptnet model in the hope that it's most suited to the codenames task. I say hope because at this point it requires further work to evaluate the various models available (read as 'stab in the dark').

## Helper Methods and Prep

In [2]:
from os import getcwd, chdir

chdir("../")  # Hate this, but quick and dirty
import pandas as pd
import numpy as np
import requests
import zipfile
import plotly.express as px
import plotly.graph_objs as go
import io
import gensim.downloader as api
import logging

from collections import namedtuple
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import Normalizer
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from itertools import combinations
from codenames.codenames_board import CodenamesGame
from tqdm.auto import tqdm
from os.path import isfile, join
from sklearn.metrics.pairwise import cosine_similarity as cosine
from operator import itemgetter

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

<IPython.core.display.Javascript object>

In [3]:
model_name = "conceptnet-numberbatch-17-06-300"
model_path = join('models', model_name)
if isfile(model_path):
    model = KeyedVectors.load(model_path)
else:
    model = api.load(
        model_name
    )
    model.save(model_path)


<IPython.core.display.Javascript object>

In [4]:
# A small adjustment to the vocab to remove all none english words and strip away the language prefix, can also be used in the future to limit vocab for the guessing agent
(
    english_only_vocab,
    english_only_vectors,
    english_only_index2entity,
    english_only_vectors_norm,
) = ({}, [], [], [])

for word, vector, index2entity in zip(
    model.vocab.keys(), model.vectors, model.index2entity
):

    if "/c/en/" in word:
        vocab = model.vocab[word]
        
        vocab.index = len(english_only_index2entity)
        english_only_vocab[word.replace("/c/en/","")] = vocab
        english_only_vectors.append(vector)
        english_only_index2entity.append(index2entity.replace("/c/en/",""))

model.vocab = english_only_vocab
model.vectors = np.array(english_only_vectors)
model.index2entity = english_only_index2entity
model.index2word = english_only_index2entity


<IPython.core.display.Javascript object>

In [5]:
def get_game_data(codenames_game, model):
    team = codenames_game.current_team
    enemy_team = "blue" if team == "red" else "blue"
    spymaster_map = codenames_game.spymaster_map
    revealed = codenames_game.revealed
    cards = [smart_lower(card, model) for card in codenames_game.codename_cards]
    in_play_cards = cards.copy()
    ally_cards = [
        c for i, c in enumerate(cards) if i in spymaster_map[team] and not revealed[i]
    ]
    ally_cards = [smart_lower(ally_card, model) for ally_card in ally_cards]
    enemy_cards = [
        c
        for i, c in enumerate(cards)
        if i in spymaster_map[enemy_team] and not revealed[i]
    ]
    enemy_cards = [smart_lower(enemy_card, model) for enemy_card in enemy_cards]
    neutral_cards = [
        c
        for i, c in enumerate(cards)
        if i in spymaster_map["neutral"] and not revealed[i]
    ]
    neutral_cards = [smart_lower(neutral_card, model) for neutral_card in neutral_cards]
    assassin_card = cards[spymaster_map["assassin"][0]]
    cards = [c for i, c in enumerate(cards) if not revealed[i]]
    return (
        cards,
        ally_cards,
        enemy_cards,
        neutral_cards,
        assassin_card,
        spymaster_map,
        team,
    )


def smart_lower(word, model):
    checked = False
    adjusted = word.replace(" ", "").lower()
    try:
        model.get_vector(adjusted)
    except KeyError:
        adjusted = adjusted.capitalize()
        model.get_vector(adjusted)

    return adjusted


def create_word_combinations_matrices(
    ally_cards, model, default_max_combo=5, one_word_clues=False
):
    max_combination = min(len(ally_cards), default_max_combo)
    ally_combinations = []
    lowest_combination = 1 if (one_word_clues or len(ally_cards) == 1) else 2
    for i in range(lowest_combination, max_combination + 1):
        ally_combinations += list(combinations(ally_cards, i))

    ally_combination_vectors = [
        [model.get_vector(word) for word in combination]
        for combination in ally_combinations
    ]
    return ally_combinations, ally_combination_vectors


def get_most_similar(
    positive_cards, all_cards, negative_cards=None, include_score=False, topn=1
):
    try:
        most_similar_words_with_scores = model.most_similar(
            positive=positive_cards, negative=negative_cards, topn=50,
        )
        possible_clue_words = []
        for most_similar_word_w_score in most_similar_words_with_scores:
            word = most_similar_word_w_score[0]
            if all([[c not in word for c in all_cards]]):
                possible_clue_words.append(word)

        return possible_clue_words[:topn]
    except IndexError:
        logger.critical(f"No valid clue found!\nCombination was {positive_cards}")
        logger.critical


def get_most_similar_vectors_for_combos(word_combinations, all_cards, **kwargs):
    most_similar_words = {
        combination: get_most_similar(
            positive_cards=list(combination), all_cards=all_cards, **kwargs
        )
        for combination in word_combinations
    }

    return most_similar_words


ClueTuple = namedtuple("ClueTuple", ["clue", "intended_combo", "board_similarities"])


def create_clue_tuples(word_combo_clue_dict, model):
    clue_tuples = []
    for clue_words, clues in word_combo_clue_dict.items():
        for clue in clues:
            clue_tuples.append(
                ClueTuple(
                    clue,
                    clue_words,
                    [(card, model.similarity(card, clue)) for card in cards],
                )
            )

    return clue_tuples


def create_clue_df(clue_tuples, cards):
    dataframe_tuples = []
    for clue, intended_combo, board_similarities in clue_tuples:
        dataframe_tuples.append(
            (
                clue,
                intended_combo,
                *[card_similarity[1] for card_similarity in board_similarities],
            )
        )

    return pd.DataFrame(
        dataframe_tuples, columns=["clue", "intended_combo"] + cards
    ).set_index(["clue", "intended_combo"])


def calculate_best_clue(
    clue_df,
    spymaster_map,
    ally_cards,
    enemy_cards,
    neutral_cards,
    assassin_card,
    **kwargs,
):
    assassin_weight = kwargs.get("assassin_weight", -10)
    enemy_weight = kwargs.get("enemy_weight", -5)
    neutral_weight = kwargs.get("neutral_weight", 0)
    ally_weight = kwargs.get("ally_weight", 10)
    risk_weight = kwargs.get("risk_weight", 0)
    clue_score_threshold = kwargs.get("clue_score_threshold", 0)
    with_normalisation = kwargs.get("with_normalisation", False)

    ally_cards_len = len(ally_cards)
    weighted_clue_df = clue_df.copy()
    if with_normalisation:
        norm = Normalizer()
        weighted_clue_df.iloc[:, :25] = norm.fit_transform(
            weighted_clue_df.iloc[:, :25]
        )
    weighted_clue_df["raw_clue_length"] = weighted_clue_df.index.get_level_values(
        "intended_combo"
    ).str.len()
    # clip scores below the threshold
    weighted_clue_df.loc[:, ally_cards] = (
        ally_df := weighted_clue_df.loc[:, ally_cards]
    ).where(ally_df >= clue_score_threshold, 0)
    # If clue word score is part of intended, but below threshold we reduce the clue length!
    weighted_clue_df["amended_combo"] = create_amended_combos(weighted_clue_df)
    weighted_clue_df["actual_combo_length"] = weighted_clue_df.amended_combo.str.len()

    # Apply various weights
    weighted_clue_df.loc[:, [assassin_card]] = (
        weighted_clue_df.loc[:, [assassin_card]] * assassin_weight
    )
    weighted_clue_df.loc[:, neutral_cards] = (
        weighted_clue_df.loc[:, neutral_cards] * neutral_weight
    )
    weighted_clue_df.loc[:, enemy_cards] = (
        weighted_clue_df.loc[:, enemy_cards] * enemy_weight
    )
    weighted_clue_df.loc[:, ally_cards] = (
        weighted_clue_df.loc[:, ally_cards] * ally_weight
    )
    weighted_clue_df["weighted_score"] = weighted_clue_df.iloc[:, :25].sum(axis=1)
    weighted_clue_df["weighted_score"] = weighted_clue_df.weighted_score + (
        weighted_clue_df.actual_combo_length * risk_weight
    )
    best_clue = weighted_clue_df.iloc[weighted_clue_df["weighted_score"].argmax()]

    return best_clue, weighted_clue_df


def restrict_vocab_with_set(w2v, restricted_word_set):
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.vocab)):
        word = w2v.index2entity[i]
        vec = w2v.vectors[i]
        vocab = w2v.vocab[word]
        #         vec_norm = w2v.vectors_norm[i]
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
    #             new_vectors_norm.append(vec_norm)

    w2v.vocab = new_vocab
    w2v.vectors = np.array(new_vectors)
    w2v.index2entity = np.array(new_index2entity)
    w2v.index2word = np.array(new_index2entity)


def create_amended_combos(df):
    #  Can be done with apply, doing it dirtily in the hope of a heavy refactor after the fact
    new_combos = []
    for idx, row in df.iterrows():
        intended_combo = idx[1]
        zero_values = row.loc[row == 0]
        new_combo = [c for c in intended_combo if c not in zero_values.index]
        new_combos.append(new_combo)
    return new_combos


def check_card_type(idx, spymaster_map, revealed):
    revealed_idxs = np.where(np.asarray(revealed) == True)[0]
    if idx not in revealed_idxs:
        return 'hidden'
    for card_type, type_idxs in spymaster_map.items():
        if idx in type_idxs:
            return card_type

<IPython.core.display.Javascript object>

## Walkthrough

In [7]:
codenames_game = CodenamesGame(words_loc="codenames/words.txt")
(
    cards,
    ally_cards,
    enemy_cards,
    neutral_cards,
    assassin_card,
    spymaster_map,
    team,
) = get_game_data(codenames_game, model)
enemy_team = "blue" if team == "red" else "red"

<IPython.core.display.Javascript object>

In [8]:
word_combos, vector_combos = create_word_combinations_matrices(ally_cards, model)
word_combo_clue_dict = get_most_similar_vectors_for_combos(
    word_combos, [assassin_card], topn=3
)
clue_tuples = create_clue_tuples(word_combo_clue_dict, model)

clue_df = create_clue_df(clue_tuples, cards)

best_clue, weighted_df = calculate_best_clue(
    clue_df,
    spymaster_map,
    enemy_colour=enemy_team,
    team_colour=team,
    assassin_weight=-10,
    enemy_weight=-1,
    neutral_weight=-1,
    ally_weight=1,
    risk_weight=0.1,
    clue_score_threshold=0.2,
)

weighted_df = weighted_df[weighted_df.actual_combo_length >= 1]
# Removing any clue duplicates (where there were multiple intended combos but the amended combos are identical)
weighted_df = (
    weighted_df.reset_index()
    .drop_duplicates(subset=["clue", "weighted_score"])
    .sort_values(["weighted_score"], ascending=False)
)
# Column reorder for readbility
weighted_df = weighted_df[
    list(weighted_df.columns[:2])
    + list(weighted_df.columns[27:])
    + list(weighted_df.columns[2:27])
]
weighted_df.head().style

/Users/josh/anaconda3/envs/codenames/lib/python3.8/site-packages/gensim/models/keyedvectors.py:2389: RuntimeWarning: invalid value encountered in true_divide
  return (m / dist).astype(REAL)


KeyboardInterrupt: 

<IPython.core.display.Javascript object>

Interesting clues come out of this, but it's obvious that the vocabulary is a little too good. We have a list of the 100k most used words in the english vocab, using this we can reduce it down somewhat. Initially, we can just take 100k and see the effect. After this there may be some room to experiment with taking different slices or randomly selecting. It should be noted that for the guessers, this will be refined, making the guessers randomly articulate.

In [6]:
vocab_lines = open("data/en_vocab_100k.txt", "r").readlines()
vocab_list = [l.replace("\n", "").lower() for l in vocab_lines if "#!" not in l]
# Need to remember codenames words!!
with open("codenames/words.txt", "r", newline="\n") as inputfile:
    codenames_words = inputfile.read().split("\n")
vocab_list += list(map(str.lower, codenames_words))
vocab_set = set(vocab_list)

<IPython.core.display.Javascript object>

In [7]:
fully_articulcated_vocab = len(model.index2entity)
# TODO check top 100k minus conceptnet to see diff
restrict_vocab_with_set(model, vocab_set)
# To get vocab size
fully_articulcated_vocab, len(model.index2entity)

(417194, 46327)

<IPython.core.display.Javascript object>

We have a much reduced vocab size - around 10% of the fully articulate model - so the results in the spymaster clue df will presumably be of a lower clue size.

In [8]:
codenames_game = CodenamesGame(words_loc="codenames/words.txt")
(
    cards,
    ally_cards,
    enemy_cards,
    neutral_cards,
    assassin_card,
    spymaster_map,
    team,
) = get_game_data(codenames_game, model)
enemy_team = "blue" if team == "red" else "red"

word_combos, vector_combos = create_word_combinations_matrices(
    ally_cards, model, default_max_combo=9
)
word_combo_clue_dict = get_most_similar_vectors_for_combos(
    word_combos, cards, negative_cards=[assassin_card], topn=3
)
clue_tuples = create_clue_tuples(word_combo_clue_dict, model)

clue_df = create_clue_df(clue_tuples, cards)

best_clue, weighted_df = calculate_best_clue(
    clue_df,
    spymaster_map,
    ally_cards,
    enemy_cards,
    neutral_cards,
    assassin_card,
    assassin_weight=-10,
    enemy_weight=-5,
    neutral_weight=-1,
    ally_weight=1,
    risk_weight=3,
    clue_score_threshold=0.2,
)

weighted_df = weighted_df[weighted_df.actual_combo_length >= 1]
# Removing any clue duplicates (where there were multiple intended combos but the amended combos are identical)
weighted_df = (
    weighted_df.reset_index()
    .drop_duplicates(subset=["clue", "weighted_score", "actual_combo_length"],)
    .sort_values(["weighted_score"], ascending=False)
)

weighted_df.reset_index(inplace=True)
# Column reorder for readbility
weighted_df = weighted_df[
    list(weighted_df.columns[:2])
    + list(weighted_df.columns[27:])
    + list(weighted_df.columns[2:27])
]
weighted_df.head().style

/Users/josh/anaconda3/envs/codenames/lib/python3.8/site-packages/gensim/models/keyedvectors.py:2389: RuntimeWarning: invalid value encountered in true_divide
  return (m / dist).astype(REAL)


,index,clue,spy,raw_clue_length,amended_combo,actual_combo_length,weighted_score,intended_combo,litter,ring,cap,pie,rock,bottle,stick,fan,ketchup,mouse,ball,pass,van,dog,pirate,cover,phoenix,ninja,robot,dinosaur,whip,pyramid,marble,mexico
0,440,lids,-0.000000,4,"['cap', 'bottle', 'cover']",3,3.349888,"('cap', 'bottle', 'mouse', 'cover')",0.229848,-0.012218,-1.732497,0.062269,-0.000000,-2.096795,-0.093886,-0.685894,-0.000000,-0.000000,0.087355,-0.055474,0.052643,-0.002764,0.011704,-1.577537,0.030649,-0.001706,0.008704,-0.085070,0.071781,0.138776,-0.000000,-0.000000
1,362,jug,-0.000000,4,"['cap', 'bottle', 'ketchup']",3,3.331888,"('cap', 'rock', 'bottle', 'ketchup')",0.131581,-0.041590,-1.034142,0.123805,-0.000000,-3.331931,-0.077329,-0.431794,-1.293153,-0.000000,0.124555,-0.012807,0.072111,-0.021658,-0.004810,-0.000000,0.047317,0.054007,-0.011446,-0.096494,0.037867,0.097801,-0.000000,-0.000000
2,430,carafe,-0.000000,4,"['cap', 'bottle', 'ketchup']",3,3.151740,"('cap', 'bottle', 'ketchup', 'marble')",0.069527,-0.009211,-1.220681,0.048070,-0.000000,-3.414998,0.018979,-0.226367,-1.138656,-0.000000,0.101839,0.014074,-0.008497,0.025272,0.004491,-0.000000,0.023625,0.019680,-0.083997,-0.137793,0.017501,0.048883,-0.000000,-0.000000
3,823,piedra,-0.000000,5,"['rock', 'marble', 'mexico']",3,3.115624,"('cap', 'rock', 'mouse', 'marble', 'mexico')",0.018252,-0.033660,-0.000000,0.030522,-2.652389,-0.000000,-0.044242,-0.407370,-0.000000,-0.000000,0.064565,0.028955,0.001675,0.032129,-0.031284,-0.000000,-0.195606,0.023210,0.059380,0.100424,-0.010960,0.247541,-1.658636,-1.456882
4,293,talavera,-0.000000,3,"['marble', 'mexico']",2,2.830036,"('bottle', 'marble', 'mexico')",-0.012927,-0.076425,-0.000000,0.206235,-0.000000,-0.000000,0.007885,0.368702,-0.000000,-0.000000,0.046880,-0.035604,0.044580,-0.024764,-0.035686,-0.000000,-0.107668,-0.116146,-0.077512,0.149664,0.017828,0.085095,-1.543653,-2.066447


<IPython.core.display.Javascript object>

In [23]:
codenames_game = CodenamesGame(words_loc="codenames/words.txt")
all_cards, _, _, _, _, _, _ = get_game_data(codenames_game, model)
assassin_weight = -1
enemy_weight = -1
neutral_weight = -1
ally_weight = 5
risk_weight = 5
clue_score_threshold = 0.3
one_word_clues = False


while not codenames_game.winning_team:
    (
        cards,
        ally_cards,
        enemy_cards,
        neutral_cards,
        assassin_card,
        spymaster_map,
        team,
    ) = get_game_data(codenames_game, model)
    draw_board(all_cards, spymaster_map, codenames_game.revealed)
    print(f"It's {team}'s turn!")
    #     print(f"{team}'s cards are {ally_cards}")
    enemy_team = "blue" if team == "red" else "red"

    word_combos, vector_combos = create_word_combinations_matrices(
        ally_cards, model, default_max_combo=9, one_word_clues=one_word_clues
    )
    word_combo_clue_dict = get_most_similar_vectors_for_combos(
        word_combos, cards, negative_cards=[assassin_card], topn=10
    )
    print(
        f"Spymaster is looking at {len(list(word_combo_clue_dict.keys()))} possible clues"
    )
    clue_tuples = create_clue_tuples(word_combo_clue_dict, model)

    clue_df = create_clue_df(clue_tuples, cards)

    best_clue, weighted_df = calculate_best_clue(
        clue_df,
        spymaster_map,
        ally_cards,
        enemy_cards,
        neutral_cards,
        assassin_card,
        assassin_weight=assassin_weight,
        enemy_weight=enemy_weight,
        neutral_weight=ally_weight,
        ally_weight=ally_weight,
        risk_weight=risk_weight,
        clue_score_threshold=clue_score_threshold,
        with_normalisation=True,
    )

    if not one_word_clues and weighted_df.actual_combo_length.max() > 1:
        weighted_df = weighted_df[weighted_df.actual_combo_length > 1]
    else:
        weighted_df = weighted_df[weighted_df.actual_combo_length >= 1]

    # Removing any clue duplicates (where there were multiple intended combos but the amended combos are identical)
    weighted_df = (
        weighted_df.reset_index()
        .drop_duplicates(subset=["clue", "weighted_score", "actual_combo_length"],)
        .sort_values(["weighted_score"], ascending=False)
    )
    weighted_df.reset_index(inplace=True)
    best_clue_idx = weighted_df.weighted_score.argmax()
    best_clue_row = weighted_df.loc[best_clue_idx]
    clue = best_clue_row["clue"]
    clue_number = best_clue_row["actual_combo_length"]
    print(f"{team} spymaster gives clue: {clue}, {clue_number}")
    remaining_cards = cards.copy()
    while (
        team == codenames_game.current_team and codenames_game.round_score < clue_number
    ):
        # If word not in vocab, none or special "I don't fucking know"
        guess = model.most_similar_to_given(clue, remaining_cards)
        codenames_game.make_guess(guess.upper())
        remaining_cards.remove(guess)
        print(f"{team} team guesses: {guess}")
        correct = codenames_game.current_team == team
        if correct:
            print(f"Correct! Round score is {codenames_game.round_score}")
        else:
            print(f"Dang! Guesser missed the mark")
        if codenames_game.round_score == clue_number:
            codenames_game.next_turn()

print(f"{codenames_game.winning_team} team won!!")

It's blue's turn!
Spymaster is looking at 502 possible clues
blue spymaster gives clue: spinner, 3
blue team guesses: cricket
Correct! Round score is 1
blue team guesses: pole
Dang! Guesser missed the mark


It's red's turn!
Spymaster is looking at 120 possible clues
red spymaster gives clue: noodles, 3
red team guesses: ketchup
Correct! Round score is 1
red team guesses: paste
Correct! Round score is 2
red team guesses: carrot
Dang! Guesser missed the mark


It's blue's turn!
Spymaster is looking at 247 possible clues
blue spymaster gives clue: chink, 3
blue team guesses: hole
Correct! Round score is 1
blue team guesses: china
Correct! Round score is 2
blue team guesses: beijing
Dang! Guesser missed the mark


It's red's turn!
Spymaster is looking at 11 possible clues
red spymaster gives clue: rabbits, 1
red team guesses: rabbit
Correct! Round score is 1


It's blue's turn!
Spymaster is looking at 57 possible clues
blue spymaster gives clue: sprung, 2
blue team guesses: spring
Correct! Round score is 1
blue team guesses: slip
Correct! Round score is 2


...

It's blue's turn!
Spymaster is looking at 11 possible clues
blue spymaster gives clue: gamblers, 2
blue team guesses: roulette
Correct! Round score is 1
blue team guesses: millionaire
Correct! Round score is 2


It's red's turn!
Spymaster is looking at 1 possible clues
red spymaster gives clue: lawn, 1
red team guesses: grass
Correct! Round score is 1


It's blue's turn!
Spymaster is looking at 1 possible clues
blue spymaster gives clue: venus, 1
blue team guesses: jupiter
Correct! Round score is 1


It's red's turn!
Spymaster is looking at 1 possible clues
red spymaster gives clue: postal, 1
red team guesses: mail
Correct! Round score is 1
red team won!!


<IPython.core.display.Javascript object>

In [21]:
ally_cards, enemy_cards, assassin_card

(['boot'], ['boot'], 'mount')

<IPython.core.display.Javascript object>

In [22]:
neutral_cards

['band', 'washer', 'back', 'pitch', 'fighter', 'log', 'fall']

<IPython.core.display.Javascript object>

In [154]:
import plotly.graph_objs.layout as layout

<IPython.core.display.Javascript object>

In [155]:
layout.Annotation

plotly.graph_objs.layout._annotation.Annotation

<IPython.core.display.Javascript object>

In [13]:
unrevealed_cards = [
    c for idx, c in enumerate(cards) if not codenames_game.revealed[idx]
]
revealed_idxs = np.where(codenames_game.revealed ==True)


def check_card_type(idx, spymaster_map, revealed):
    revealed_idxs = np.where(np.asarray(revealed) == True)[0]
    if idx not in revealed_idxs:
        return 'hidden'
    for card_type, type_idxs in spymaster_map.items():
        if idx in type_idxs:
            return card_type
        


<IPython.core.display.Javascript object>

In [90]:
np.where(codenames_game.revealed ==True)


(array([], dtype=int64),)

<IPython.core.display.Javascript object>

In [16]:
colour_map = {
    "neutral": [194, 183, 54, 0.25],
    "hidden": [246, 232, 204, 0.25],
    "assassin": [0, 0, 0, 0.25],
    "red": [213, 64, 8, 0.25],
    "blue": [57, 105, 145, 0.25],
}

<IPython.core.display.Javascript object>

In [101]:
split_cards = np.array_split(cards, 5)
split_card_colours = np.array_split(
    [
        colour_map[
            check_card_type(idx, codenames_game.spymaster_map, codenames_game.revealed)
        ]
        for idx, c in enumerate(cards)
    ],
    5,
)

<ipython-input-95-6e1f991e57d9>:10: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.



<IPython.core.display.Javascript object>

In [102]:
split_card_colours

[array([[246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25]]),
 array([[246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25]]),
 array([[246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25]]),
 array([[246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25]]),
 array([[246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 204.  ,   0.25],
        [246.  , 232.  , 2

<IPython.core.display.Javascript object>

In [158]:
import plotly.express as px
import plotly.graph_objs as go
import plotly.graph_objs.layout as layout

# annotations = layout.Annotations()
fig = px.imshow(split_card_colours)
for y_idx, row in enumerate(split_cards):
    for x_idx, card in enumerate(row):
        fig.add_annotation(text=card, x=x_idx, y=y_idx, showarrow=False)

fig["layout"].update(annotations=annotations)
fig.show()

<IPython.core.display.Javascript object>

In [14]:
def draw_board(cards, spymaster_map, revealed):
    split_cards = np.array_split(cards, 5)
    split_card_colours = np.array_split(
        [
            colour_map[
                check_card_type(
                    idx, codenames_game.spymaster_map, codenames_game.revealed
                )
            ]
            for idx, c in enumerate(cards)
        ],
        5,
    )
    fig = px.imshow(split_card_colours)
    for y_idx, row in enumerate(split_cards):
        for x_idx, card in enumerate(row):
            fig.add_annotation(text=card, x=x_idx, y=y_idx, showarrow=False)
    fig.show()

<IPython.core.display.Javascript object>

In [150]:
draw_board(all_cards, codenames_game.spymaster_map, codenames_game.revealed)

<IPython.core.display.Javascript object>

In [133]:
check_card_type(1, codenames_game.spymaster_map, codenames_game.revealed)

[True, True, True, True, False, True, True, False, False, False, False, True, True, True, False, True, False, True, True, True, True, False, True, False, True]
(array([], dtype=int64),)


<ipython-input-131-c53f44804feb>:11: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.



'hidden'

<IPython.core.display.Javascript object>

In [144]:
model.most

array([ 0,  1,  2,  3,  5,  6, 11, 12, 13, 15, 17, 18, 19, 20, 22, 24])

<IPython.core.display.Javascript object>